## Hamiltonian Construction for Small Molecules

This notebook constructs and analyzes qubit Hamiltonians for small molecules using PySCF and Qiskit Nature. We generate Hamiltonians for H₂, LiH, and HeH⁺ molecules as functions of bond length using the STO-3G basis and the Jordan-Wigner mapping.

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
import numpy as np

def h2_qubit_hamiltonian(R):
    geometry = f"H 0 0 0; H 0 0 {R}"

    driver = PySCFDriver(
        atom=geometry,
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    problem = driver.run()

    fermionic_op = problem.hamiltonian.second_q_op()

    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)

    return qubit_op, problem


## Electronic Structure Problem Setup

The PySCFDriver generates an `ElectronicStructureProblem` containing the electronic Hamiltonian. The key component is the `ElectronicEnergy` operator, which encodes the molecular Hamiltonian from:
- **One-body integrals**: Kinetic energy and electron-nucleus interactions
- **Two-body integrals**: Electron-electron repulsion interactions

These integrals are computed classically by PySCF and then mapped to a qubit representation via the Jordan-Wigner transformation.

## Exact Ground State Energy Reference

For each bond length R, we compute the exact ground state energy using classical diagonalization:

1. **Hamiltonian generation**: Generate electronic Hamiltonian using PySCF in the STO-3G basis
2. **Fermionic-to-qubit mapping**: Apply Jordan-Wigner transformation to map fermionic operators to Pauli operators
3. **Classical diagonalization**: Solve exactly by diagonalizing the qubit Hamiltonian matrix
4. **Nuclear repulsion**: Add classical nuclear repulsion energy to obtain total energy

These reference energies provide a baseline for comparing variational quantum algorithms.

In [2]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper


In [ ]:

def exact_energy(qubit_op, problem):
    # Convert SparsePauliOp → dense matrix
    H = qubit_op.to_matrix()

    # Exact diagonalization
    eigvals = np.linalg.eigvalsh(H)
    energy = np.min(eigvals).real

    # Add nuclear repulsion
    energy += problem.nuclear_repulsion_energy

    return energy


In [ ]:

Rs = np.linspace(0.3, 2.0, 12)

energies = []


print("H_2 values are")
for R in Rs:
    qubit_op, problem = h2_qubit_hamiltonian(R)
    E0 = exact_energy(qubit_op, problem)
    energies.append(E0)
    print(f"R = {R:.2f} Å  |  E₀ = {E0:.6f} Ha")



R = 0.30 Å  |  E₀ = -0.601804 Ha
R = 0.45 Å  |  E₀ = -1.004561 Ha
R = 0.61 Å  |  E₀ = -1.119418 Ha
R = 0.76 Å  |  E₀ = -1.136642 Ha
R = 0.92 Å  |  E₀ = -1.117345 Ha
R = 1.07 Å  |  E₀ = -1.085302 Ha
R = 1.23 Å  |  E₀ = -1.050726 Ha
R = 1.38 Å  |  E₀ = -1.018887 Ha
R = 1.54 Å  |  E₀ = -0.992503 Ha
R = 1.69 Å  |  E₀ = -0.972417 Ha
R = 1.85 Å  |  E₀ = -0.958174 Ha
R = 2.00 Å  |  E₀ = -0.948641 Ha


## Extending to Larger Molecules: LiH

While H₂ serves as a simple benchmark, we also study larger molecules to demonstrate scalability. **Lithium hydride (LiH)** is chosen for the following reasons:

- **H₂ complexity**: 2 qubits (Hilbert space dimension: $2^2 = 4$)
- **LiH complexity**: 12 qubits (Hilbert space dimension: $2^{12} = 4096$)

With the larger system, dense matrix diagonalization becomes impractical. Instead, we use sparse eigenvalue solvers (scipy's `eigsh`) which compute only the ground state without requiring full diagonalization.

## Historical Benchmark: HeH⁺

As a third system, we examine **helium hydride cation (HeH⁺)**:
- One of the simplest diatomic molecules with only 2 electrons
- Non-degenerate ground state (no symmetry-induced degeneracy)
- **Historical significance**: The first molecule to form after the Big Bang during cosmic recombination (~380,000 years post-Big Bang)
- Qubit requirement: 4 qubits

In [ ]:
def LiH_qubit_hamiltonian(R):
    geometry = f"Li 0 0 0; H 0 0 {R}"

    driver = PySCFDriver(
        atom=geometry,
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    problem = driver.run()

    fermionic_op = problem.hamiltonian.second_q_op()

    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)

    return qubit_op, problem

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh
def exact_energy_sparse(qubit_op, problem):

    # Convert to sparse matrix
    H = csc_matrix(qubit_op.to_matrix())
    
    E0 = eigsh(H, k=1, which='SA', return_eigenvectors=False)[0].real
    
    E0 += problem.nuclear_repulsion_energy
    return E0


Rs = np.linspace(0.3, 2.0, 12)

energies = []

print("LiH values are")
for R in Rs:
    qubit_op, problem = LiH_qubit_hamiltonian(R)
    E0 = exact_energy_sparse(qubit_op, problem)
    energies.append(E0)
    print(f"R = {R:.2f} Å  |  E₀ = {E0:.6f} Ha")

R = 0.30 Å  |  E₀ = -5.881787 Ha
R = 0.45 Å  |  E₀ = -6.887721 Ha
R = 0.61 Å  |  E₀ = -7.339137 Ha
R = 0.76 Å  |  E₀ = -7.592572 Ha
R = 0.92 Å  |  E₀ = -7.736339 Ha
R = 1.07 Å  |  E₀ = -7.815946 Ha
R = 1.23 Å  |  E₀ = -7.857863 Ha
R = 1.38 Å  |  E₀ = -7.877212 Ha
R = 1.54 Å  |  E₀ = -7.882741 Ha
R = 1.69 Å  |  E₀ = -7.879791 Ha
R = 1.85 Å  |  E₀ = -7.871805 Ha
R = 2.00 Å  |  E₀ = -7.861088 Ha


In [ ]:
def HeH_qubit_hamiltonian(R):
    geometry = f"He 0 0 0; H 0 0 {R}"

    driver = PySCFDriver(
        atom=geometry,
        basis="sto3g",
        charge=1,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    problem = driver.run()

    fermionic_op = problem.hamiltonian.second_q_op()

    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)

    return qubit_op, problem

def exact_energy(qubit_op, problem):
    # Convert SparsePauliOp → dense matrix
    H = qubit_op.to_matrix()

    # Exact diagonalization
    eigvals = np.linalg.eigvalsh(H)
    energy = np.min(eigvals).real

    # Add nuclear repulsion
    energy += problem.nuclear_repulsion_energy

    return energy


Rs = np.linspace(0.3, 2.0, 12)

energies = []
print("HeH+ Values are:")
for R in Rs:
    qubit_op, problem = HeH_qubit_hamiltonian(R)
    E0 = exact_energy(qubit_op, problem)
    energies.append(E0)
    print(f"R = {R:.2f} Å  |  E₀ = {E0:.6f} Ha")

R = 0.30 Å  |  E₀ = -1.799073 Ha
R = 0.45 Å  |  E₀ = -2.542515 Ha
R = 0.61 Å  |  E₀ = -2.790101 Ha
R = 0.76 Å  |  E₀ = -3.005387 Ha
R = 0.92 Å  |  E₀ = -3.119441 Ha
R = 1.07 Å  |  E₀ = -3.183798 Ha
R = 1.23 Å  |  E₀ = -3.221369 Ha
R = 1.38 Å  |  E₀ = -3.243581 Ha
R = 1.54 Å  |  E₀ = -3.256716 Ha
R = 1.69 Å  |  E₀ = -3.264414 Ha
R = 1.85 Å  |  E₀ = -3.268854 Ha
R = 2.00 Å  |  E₀ = -3.271368 Ha


## Hamiltonian Representation in Pauli Basis

After Jordan-Wigner mapping, the electronic Hamiltonian is expressed as a sum of Pauli operators. The cells below display each term in the Pauli basis (Z, X, Y, I identities on each qubit) with their corresponding coefficients.

In [11]:
qubit_op_h2, problem = h2_qubit_hamiltonian(R)

for pauli, coeff in zip(qubit_op_h2.paulis, qubit_op.coeffs):
    print(pauli.to_label(), coeff)


IIII (-2.465154446445153+0j)
IIIZ (0.7009770916216935+0j)
IIZI (0.17073256536616271+0j)
IIZZ (0.06567645923909367+0j)
IZII (0.7009770916216935+0j)
IZIZ (0.26093579771720327+0j)
ZIII (0.02796578602819912+0j)
ZIIZ (0.018431746888127796+0j)
YYYY (0.02796578602819912+0j)
XXYY (0.018431746888127796+0j)
YYXX (0.17073256536616266+0j)
XXXX (0.06832221057522317+0j)
IZZI (0.02796578602819912+0j)
ZIZI (0.018431746888127796+0j)
ZZII (0.02796578602819912+0j)


In [12]:
qubit_op_LiH, problem = LiH_qubit_hamiltonian(R)

print("LiH Hamiltonian:")
for pauli, coeff in zip(qubit_op_LiH.paulis, qubit_op.coeffs):
    print(pauli.to_label(), coeff)

LiH Hamiltonian:
IIIIIIIIIIII (-2.465154446445153+0j)
IIIIIIIIIIIZ (0.7009770916216935+0j)
IIIIIIIIIIZI (0.17073256536616271+0j)
IIIIIIIIIIZZ (0.06567645923909367+0j)
IIIIIIIIIYYI (0.7009770916216935+0j)
IIIIIIIIIYYZ (0.26093579771720327+0j)
IIIIIIIIIXXI (0.02796578602819912+0j)
IIIIIIIIIXXZ (0.018431746888127796+0j)
IIIIIIYZZZYI (0.02796578602819912+0j)
IIIIIIYZZZYZ (0.018431746888127796+0j)
IIIIIIXZZZXI (0.17073256536616266+0j)
IIIIIIXZZZXZ (0.06832221057522317+0j)
IIIIIIIIIZII (0.02796578602819912+0j)
IIIIIIIIIZIZ (0.018431746888127796+0j)
IIIIIIYZZYII (0.02796578602819912+0j)
IIIIIIYZZYIZ (0.018431746888127796+0j)
IIIIIIXZZXII (0.0026457513361295014+0j)
IIIIIIXZZXIZ (0.0026457513361295014+0j)
IIIIIIIIZIII (0.0026457513361295014+0j)
IIIIIIIIZIIZ (0.0026457513361295014+0j)
IIIIIIIZIIII (-0.009533498947502397+0j)
IIIIIIIZIIIZ (-0.009533498947502397+0j)
IIIIIIZIIIII (0.06832221057522317+0j)
IIIIIIZIIIIZ (-0.009533498947502397+0j)
IIIIIZIIIIII (-0.009533498947502397+0j)
IIIIIZIIIIIZ (0.

In [13]:
qubit_op_HeH, problem = HeH_qubit_hamiltonian(R)

print("HeH+ Hamiltonian:")
for pauli, coeff in zip(qubit_op_HeH.paulis, qubit_op.coeffs):
    print(pauli.to_label(), coeff)

HeH+ Hamiltonian:
IIII (-2.465154446445153+0j)
IIIZ (0.7009770916216935+0j)
IIZI (0.17073256536616271+0j)
IIZZ (0.06567645923909367+0j)
IZII (0.7009770916216935+0j)
IZIZ (0.26093579771720327+0j)
YYII (0.02796578602819912+0j)
YYIZ (0.018431746888127796+0j)
XXII (0.02796578602819912+0j)
XXIZ (0.018431746888127796+0j)
ZIII (0.17073256536616266+0j)
ZIIZ (0.06832221057522317+0j)
IIYY (0.02796578602819912+0j)
IZYY (0.018431746888127796+0j)
IIXX (0.02796578602819912+0j)
IZXX (0.018431746888127796+0j)
YYYY (0.0026457513361295014+0j)
XXYY (0.0026457513361295014+0j)
YYXX (0.0026457513361295014+0j)
XXXX (0.0026457513361295014+0j)
ZIYY (-0.009533498947502397+0j)
ZIXX (-0.009533498947502397+0j)
IZZI (0.06832221057522317+0j)
YYZI (-0.009533498947502397+0j)
XXZI (-0.009533498947502397+0j)
ZIZI (0.19280199512360402+0j)
ZZII (0.06567645923909367+0j)
